<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/Project/SamVerse/Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
charts = "/content/drive/MyDrive/Colab Notebooks/charts.csv"

## Data Preparation and Cleaning

> We will be begin our data cleansing using these three methods

*   Importing data from Google Drive
*   Removing null data
*   Removng duplicates
*   List item





Import Data

**1. Pandas**

In [3]:
import pandas as pd
import numpy as np

In [4]:
%%time
df = pd.read_csv(charts)
df.head(5)

CPU times: user 53.3 s, sys: 6.62 s, total: 59.9 s
Wall time: 1min 19s


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


**2. PySpark**

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=57bd2b86c33bb66121550225183e428652c3ae1159b190e4e87ac6aa80bd6dfb
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
spark = SparkSession.builder.getOrCreate()

In [ ]:
%%time
df_py = spark.read.csv(charts, header = True)
df_py.show(5)

AnalysisException: ignored

**3. Koalas**

In [ ]:
!pip install koalas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.8/390.8 KB 11.6 MB/s eta 0:00:00


In [ ]:
import databricks.koalas as ks

In [ ]:
%%time
df_ks = ks.read_csv(charts)
df_ks.head(5)

AnalysisException: ignored

Removing null

**1. Pandas**

In [5]:
df.isnull().sum()

title           11
rank             0
date             0
artist          18
url              0
region           0
chart            0
trend            0
streams    5851610
dtype: int64

In [6]:
%%time
df = df.dropna()

CPU times: user 11.1 s, sys: 653 ms, total: 11.7 s
Wall time: 11.7 s


In [7]:
df.isnull().sum()

title      0
rank       0
date       0
artist     0
url        0
region     0
chart      0
trend      0
streams    0
dtype: int64

**2. Pyspark**

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
df_py1 = df_py.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df_py1.show()

In [ ]:
%%time
df_py = df_py.dropna()

In [ ]:
df_py1 = df_py.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df_py1.show()

**3. Koalas**

In [ ]:
df_ks.isnull().sum()

In [ ]:
%time
df_ks.dropna(how='all', inplace = True) 

In [ ]:
df_ks.isnull().sum()

Removing duplicates

**1. Pandas**

In [8]:
%time
df = df.drop_duplicates()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs


**2. Pyspark**

In [ ]:
%time
df_py.dropDuplicates()

**3. Koalas**

In [ ]:
%time
df_ks.drop_duplicates().head(5)

##Perform exploratory analysis & visualization